In [ ]:
# imports
from IPython.display import display, HTML
# from "container/card_extractor" import CardExtractor
from google.cloud import spanner
import os
import copy
import json
from shapely import geometry
from shapely.geometry import Point
from matplotlib import pyplot as plt
import random

In [ ]:
# fetching database
spanner_client = spanner.Client()
spanner_instance_id = 'tfgen-spanid-20220525101635457'
spanner_database_id = 'metadata'
instance = spanner_client.instance(spanner_instance_id)
database = instance.database(spanner_database_id)

In [ ]:
# initialise india, district shapes
with open('../container/testdata/india_shape.geojson', 'r') as file:
  india_gj_raw = file.read()
with open('../container/testdata/district_shape.geojson', 'r') as file:
  district_gj_raw = file.read()

india_gj = json.loads(india_gj_raw)
india_shape = geometry.shape(india_gj['features'][0]['geometry'])

district_gj = json.loads(district_gj_raw)
district_shape = {}
for district in district_gj['features']:
  district_shape[district['properties']['censuscode']] = geometry.shape(district['geometry'])

In [ ]:
# plot district boundaries

fig = plt.figure()
fig.set_figheight(30)
fig.set_figwidth(30)

for district in district_gj['features']:
  # if district['properties']['censuscode'] != 2:
  #   continue
  district_shape[district['properties']['censuscode']] = geometry.shape(district['geometry'])
  if type(district_shape[district['properties']['censuscode']]) is geometry.polygon.Polygon:
    x, y = district_shape[district['properties']['censuscode']].exterior.xy
    plt.plot(x, y)
  else:
    for geom in district_shape[district['properties']['censuscode']].geoms:
      x, y = geom.exterior.xy
      plt.plot(x, y)

for geom in india_shape.geoms:
  x, y = geom.exterior.xy
  plt.plot(x, y)

plt.show()

In [ ]:
# prints all states present in shapely file
st_nm = set()
for district in district_gj['features']:
  st_nm.add(district['properties']['ST_NM'])
print(len(st_nm))
st_nm = list(st_nm)
st_nm.sort()

for x in st_nm:
  print(x)

In [ ]:
# get all district ids
with database.snapshot() as snapshot:
  district_ids = snapshot.execute_sql(
    """SELECT districtid
    FROM districts
    GROUP BY 1
    ORDER BY 1
    """
  )

district_ids = list(district_ids)
district_ids = [x[0] for x in district_ids] 

In [ ]:
# get all cards with geoposition
with database.snapshot() as snapshot:
  points = snapshot.execute_sql(
    """SELECT districtid, latitude, longitude
    FROM Cards_info
    WHERE latitude is not null
    ORDER BY 1"""
  )

points = list(points)

In [ ]:
# organise
print(f'count: {len(points)}')

points_district = {}
for x in district_ids:
  points_district[x] = []

for point in points:
  points_district[point[0]].append(point[1:])

In [ ]:
# district wise extracted card count
district_card_count = {x:0 for x in district_ids}

with database.snapshot() as snapshot:
  data1 = snapshot.execute_sql(
    """SELECT districtid, COUNT(*)
    FROM Cards_info
    GROUP BY 1
    ORDER BY 1
    """
  )
data1 = list(data1)

for x in data1:
  district_card_count[x[0]] = x[1]

In [ ]:
table = {}

# stateid, district name
with database.snapshot() as snapshot:
  data = snapshot.execute_sql(
    """SELECT DistrictId, StateId, Name 
    FROM Districts
    ORDER BY 1
    """
  )
data = list(data)
print(len(data))
for x in data:
  table[x[0]] = {}
  table[x[0]]['stateid'] = x[1]
  table[x[0]]['district'] = x[2]

# state name
with database.snapshot() as snapshot:
  data = snapshot.execute_sql(
    """SELECT StateId, Name
    FROM States"""
  )
data = list(data)
data = {x[0]: x[1] for x in data}
for k in table.keys():
  table[k]['state'] = data[table[k]['stateid']]

# total cards
with database.snapshot() as snapshot:
  data = snapshot.execute_sql(
    """SELECT districtid, COUNT(*)
    FROM Cards
    GROUP BY 1"""
  )
data = list(data)
data = {x[0]: x[1] for x in data}
for k in table.keys():
  table[k]['num_cards'] = data.get(k, 0)

# total scraped cards
with database.snapshot() as snapshot:
  data = snapshot.execute_sql(
    """SELECT districtid, COUNT(*)
    FROM Cards
    WHERE ingested is true
    GROUP BY 1"""
  )
data = list(data)
data = {x[0]: x[1] for x in data}
for k in table.keys():
  table[k]['num_scraped'] = data.get(k, 0)

# total extracted cards
with database.snapshot() as snapshot:
  data = snapshot.execute_sql(
    """SELECT districtid, COUNT(*)
    FROM Cards_info
    GROUP BY 1"""
  )
data = list(data)
data = {x[0]: x[1] for x in data}
for k in table.keys():
  table[k]['num_extracted'] = data.get(k, 0)

# total extracted cards with geopos available
with database.snapshot() as snapshot:
  data = snapshot.execute_sql(
    """SELECT districtid, COUNT(*)
    FROM Cards_info
    WHERE latitude is not null and longitude is not null
    GROUP BY 1"""
  )
data = list(data)
data = {x[0]: x[1] for x in data}
for k in table.keys():
  table[k]['geopos_present'] = data.get(k, 0)

In [ ]:
paddings = [0,2,5,10,20,50]

def check_in_district(lat, long, districtid):
  if not district_shape.get(districtid):
    return len(paddings)
  
  pnt = Point(long, lat)
  if district_shape[districtid].contains(pnt):
    return 0
  else:
    dist = pnt.distance(district_shape[districtid])*111
    for i in range(1, len(paddings)):
      if dist < paddings[i]:
        return i
  
  return len(paddings)

In [ ]:
# check if within district - padding 0
in_district_pad_0 = {}
for k, v in points_district.items():
  if not district_shape.get(k):
    in_district_pad_0[k] = 0
  else:
    cnt = 0
    for point in v:
      if district_shape[k].contains(Point(point[1], point[0])):
        cnt+=1

    in_district_pad_0[k] = cnt

  cnt = in_district_pad_0[k]
  total_cnt = max(len(v), 1)
  print(f'district {k}: {cnt}/{total_cnt} - {round(cnt/total_cnt*100, 2)}%')   


In [ ]:
# check if within district - padding 0,2,5,10,20,50 km
paddings = [0,2,5,10,20,50]
in_district = [{k:0 for k in points_district} for _ in range(len(paddings))]
for k, v in points_district.items():
  if not district_shape.get(k):
    continue
  else:
    for point in v:
      pnt = Point(point[1], point[0])
      if district_shape[k].contains(pnt):
        in_district[0][k] += 1
      else:
        dist = pnt.distance(district_shape[k])*111
        for i in range(1, len(paddings)):
          if dist < paddings[i]:
            in_district[i][k] += 1
            break
      
    for i in range(1, len(paddings)):
      in_district[i][k] += in_district[i-1][k]

  print(f'{k}:\t{len(v)}\t{in_district[0][k]}\t{in_district[1][k]}\t{in_district[2][k]}\t{in_district[3][k]}\t{in_district[4][k]}\t{in_district[5][k]}')

In [ ]:
# display table
for k, v in table.items():
  print(f'{k}\t{v["stateid"]}\t{v["district"]}\t{v["state"]}\t{v["num_cards"]}\t{v["num_scraped"]}\t{v["num_extracted"]}'
    f'\t{v["geopos_present"]}\t{in_district[0].get(k, 0)}\t{in_district[1].get(k, 0)}\t{in_district[2].get(k, 0)}\t{in_district[3].get(k, 0)}'
    f'\t{in_district[4].get(k, 0)}\t{in_district[5].get(k, 0)}'
  )

In [ ]:
table2 = {}

temp = {'state':'', 'num_cards':0, 'num_scraped':0, 'num_extracted':0, 'geopos_present':0, 'in_district':[0 for _ in range(len(paddings))]}

for k, v in table.items():
  sid = v['stateid']
  if not table2.get(sid):
    table2[sid] = copy.deepcopy(temp)

  table2[sid]['state'] = v['state']
  table2[sid]['num_cards'] += v['num_cards']
  table2[sid]['num_scraped'] += v['num_scraped']
  table2[sid]['num_extracted'] += v['num_extracted']
  table2[sid]['geopos_present'] += v['geopos_present']
  for i in range(len(paddings)):
    table2[sid]['in_district'][i] += in_district[i].get(k,0)

sids = list(table2.keys())
sids.sort()
for sid in sids:
  print(f"{sid}\t{table2[sid]['state']}\t{table2[sid]['num_cards']}\t{table2[sid]['num_scraped']}\t{table2[sid]['num_extracted']}\t{table2[sid]['geopos_present']}\t{table2[sid]['in_district'][0]}\t{table2[sid]['in_district'][1]}\t{table2[sid]['in_district'][2]}\t{table2[sid]['in_district'][3]}\t{table2[sid]['in_district'][4]}\t{table2[sid]['in_district'][5]}")
  

In [ ]:
# labwise district check
table3 = {}

# num extracted
with database.snapshot() as snapshot:
  labs = snapshot.execute_sql(
    """SELECT soil_test_lab, COUNT(*)
    FROM Cards_info
    GROUP BY 1
    ORDER BY 1
    """
  )
labs = list(labs)
for x in labs:
  table3[x[0]] = {}
  table3[x[0]]['num_extracted'] = x[1]

with database.snapshot() as snapshot:
  labs = snapshot.execute_sql(
    """SELECT soil_test_lab, COUNT(*)
    FROM Cards_info
    WHERE latitude is not null
    GROUP BY 1
    ORDER BY 1
    """
  )
labs = list(labs)
for x in labs:
  table3[x[0]]['geopos_present'] = x[1]

In [ ]:
# labwise district check

with database.snapshot() as snapshot:
  labs = snapshot.execute_sql(
    """SELECT soil_test_lab, districtid, latitude, longitude
    FROM Cards_info
    WHERE latitude is not null
    ORDER BY 1
    """
  )

labs = list(labs)
print(len(labs))

In [ ]:
curr = 'start'
for lab in labs:
  if not table3[lab[0]].get('in_district'):
    print(f'{curr}: {table3.get(curr, "not found")}')
    print(f'checking for {lab[0]}')
    curr = lab[0]
    table3[lab[0]]['in_district'] = [0 for _ in range(len(paddings))]
  
  x = check_in_district(lab[2], lab[3], lab[1])
  for i in range(x, len(paddings)):
    table3[lab[0]]['in_district'][i] += 1

In [ ]:
print(len(table3.keys()))

In [ ]:
for k, v in table3.items():
  if not table3[k].get('geopos_present'):
    table3[k]['geopos_present'] = 0
    table3[k]['in_district'] = [0 for _ in range(len(paddings))]
  print(f"{k}\t{v['num_extracted']}\t{v['geopos_present']}\t{v['in_district'][0]}\t{v['in_district'][0]/max(v['geopos_present'], 1)}\t{v['in_district'][1]}\t{v['in_district'][1]/max(v['geopos_present'], 1)}\t{v['in_district'][2]}\t{v['in_district'][2]/max(v['geopos_present'], 1)}\t{v['in_district'][3]}\t{v['in_district'][3]/max(v['geopos_present'], 1)}\t{v['in_district'][4]}\t{v['in_district'][4]/max(v['geopos_present'], 1)}\t{v['in_district'][5]}\t{v['in_district'][5]/max(v['geopos_present'], 1)}")